In [3]:
import pandas as pd
import numpy as np
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import math
import h5py

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from shutil import copyfile
from copy import deepcopy

from code_base.utils import write_json, load_json

# 2024 Scored Data DF

In [2]:
train_metadata = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_prev_comp = pd.read_csv("/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV2_scored_meta_prev_comps_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_xeno_canto = pd.read_csv("/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_nodupl_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})

In [4]:
scored_birds = load_json("/home/vova/data/exps/birdclef_2024/scored_birds/sb_2024.json")

In [9]:
def extract_sampler_col(row, scored_birds_):
    if row["primary_label"] in scored_birds_:
        return row["primary_label"]
    else:
        return list(set(row["secondary_labels"]) & set(scored_birds_))[0]

In [10]:
train_metadata_prev_comp["stratify_col"] = train_metadata_prev_comp.apply(
    lambda row: extract_sampler_col(row, scored_birds),
    axis=1
)

In [21]:
train_metadata["stratify_col"] = train_metadata["primary_label"]
train_metadata_xeno_canto["stratify_col"] = train_metadata_xeno_canto["primary_label"]

In [13]:
train_metadata_prev_comp["stratify_col"].apply(lambda x: x in scored_birds).all()

True

In [15]:
# train_metadata_prev_comp.to_csv(
#     "/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV2_scored_meta_prev_comps_extended.csv",
#     index=False
# )

In [22]:
# train_metadata.to_csv(
#     "/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended.csv",
#     index=False
# )

In [23]:
# train_metadata_xeno_canto.to_csv(
#     "/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_nodupl_extended.csv",
#     index=False
# )

In [16]:
all_labels = pd.concat([
    train_metadata["primary_label"], train_metadata_prev_comp["stratify_col"], train_metadata_xeno_canto["primary_label"]
]).reset_index(drop=True)

In [17]:
sample_weights = (all_labels.value_counts() / all_labels.value_counts().sum())  ** (-0.5)

In [19]:
sample_weights

houspa      3.991019
eaywag1     4.494504
commoo3     4.715205
eurcoo      4.822198
barswa      5.140697
             ...    
integr     67.526426
blaeag1    67.526426
wbbfly1    74.653273
niwpig1    79.181753
asiope1    84.648855
Name: count, Length: 182, dtype: float64

In [20]:
write_json("/home/vova/data/exps/birdclef_2024/sample_weights/sw_2024_add_data_v1.json", sample_weights.to_dict())